<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/118_%E9%AB%98%E6%B0%B4%E6%BA%96%E3%81%AE%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E6%93%8D%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

高水準のファイル操作
====================

tempfile
--------

標準ライブラリの `tempfile` モジュールは、一時ファイルや一時ディレクトリを作成する機能を提供する。

### TemporaryFile ###

``` python
tempfile.TemporaryFile(mode='w+b', buffering=-1, encoding=None, newline=None, suffix=None, prefix=None, dir=None, *, errors=None)
```

この関数は、一時ファイルを作成し、ファイルオブジェクトを返す。ファイルは、可能な限り最も安全な手段で作成される。すなわち、ファイル名が既存のファイルと競合しないことが保証され、また、作成したユーザのみファイルを読み書き可能なパーミッション設定が行われる。ファイルオブジェクトが閉じられると、即座にファイルが自動的に削除される。

一般のファイルオブジェクトと同じで、この関数が返すファイルオブジェクトもコンテキストマネージャーなので、with 文の中で使用することができる。

引数 `mode`、`buffering`、`encoding`、`newline`、`errors` は、`open()` に対する引数として解釈される。残りの引数については次のとおり。

| 引数 | 意味 |
|:---|:---|
| `suffix` | `None` でない場合、ファイル名はその接尾辞で終わる |
| `prefix` | `None` でない場合、ファイル名はその接頭辞で始まる |
| `dir` | `None` でない場合、ファイルはそのディレクトリ下に作成される。`None` の場合、`TMPDIR`、`TEMP`、または `TMP` 環境変数の値が使われる |

In [ ]:
import tempfile

file_name = ""
with tempfile.TemporaryFile() as f:
    file_name = f.name
    f.write(b"abcd")  # デフォルトではバイナリモード
    f.seek(0)
    print(f"{f.read()=}")

print(f"{file_name=}")  # Unix ではファイル名が不可視。Windows ではファイルパスを得る

f.read()=b'abcd'
file_name=42


### NamedTemporaryFile ###

``` python
tempfile.NamedTemporaryFile(mode='w+b', buffering=-1, encoding=None, newline=None, suffix=None, prefix=None, dir=None, delete=True, *, errors=None)
```

この関数も、一時ファイルを作成し、ファイルオブジェクトを返すが、以下の点で `tempfile.TemporaryFile()` と異なる。

  * ファイルオブジェクトの `name` 属性で一時ファイルのパスが得られる。
  * `delete` 引数が追加され、これが`False` の場合、ファイルオブジェクトを閉じてもファイルが自動的には削除されない。

なお、Python 3.12 で、`delete_on_close` キーワード専用引数が追加された。`delete` が `True` で、`delete_on_close` が `False` の場合、with 文で作成された一時ファイルは with 文が終わったときにのみ削除され、途中でファイルオブジェクトが閉じられてもファイルは削除されないようになった。`delete_on_close` のデフォルトは `True`。

In [ ]:
import tempfile
from pathlib import Path

path = None
with tempfile.NamedTemporaryFile() as f:
    f.write(b'abcd')
    f.seek(0)
    print(f.read())
    path = f.name
    print(f"{path=}")
    assert Path(path).exists()

assert not Path(path).exists()

b'abcd'
path='/tmp/tmpmhpnqe0e'


In [ ]:
import tempfile
from pathlib import Path

path = None
with tempfile.NamedTemporaryFile(delete=False) as f:
    f.write(b'abcd')
    f.seek(0)
    print(f.read())
    path = f.name
    print(f"{path=}")
    assert Path(path).exists()

assert Path(path).exists()
Path(path).unlink()
assert not Path(path).exists()

b'abcd'
path='/tmp/tmpeewcl17_'


### SpooledTemporaryFile ###

``` python
tempfile.SpooledTemporaryFile(max_size=0, mode='w+b', buffering=-1, encoding=None, newline=None, suffix=None, prefix=None, dir=None, *, errors=None)
```

これは、`SpooledTemporaryFile` クラスのコンストラクタである。 `SpooledTemporaryFile` クラスは、データを基本的にメモリ上に書き出す。ファイルサイズが `max_size` を超えるか、ファイルの `fileno()` メソッドが呼び出されるとディスク上に書き出すため、メモリ消費量が増えすぎるということがない。ディスクに書き出したあとの動作は `TemporaryFile()` と同様である。

返されるオブジェクトは、`rollover()` メソッドを持ち、これにより、ファイルはサイズに関係なくディスク上のファイルにロールオーバーされる。

### TemporaryDirectory ###

``` python
tempfile.TemporaryDirectory(suffix=None, prefix=None, dir=None, ignore_cleanup_errors=False, *, delete=True)
```

これは、`TemporaryDirectory` クラスのコンストラクタであり、一時ディレクトリを作成する。`TemporaryDirectory` は、コンテキストマネージャーの機能をサポートしていて、with 文のブロックの完了時に一時ディレクトリを中身のファイルごと削除することができる。

| 引数 | 意味 |
|:---|:---|
| `suffix` | `None` でない場合、一時ディレクトリ名はその接尾辞で終わる |
| `prefix` | `None` でない場合、一時ディレクトリ名はその接頭辞で始まる |
| `dir` | `None` でない場合、一時ディレクトリは `dir` のサブディレクトリとして作成される。`None` の場合、`TMPDIR`、`TEMP`、または `TMP` 環境変数の値が使われる |
| `ignore_cleanup_errors` | `True` の場合、一時ディレクトリの削除中のエラーを無視する |
| `delete` | `True`（デフォルト）の場合、with のブロックの完了時に一時ディレクトリが削除される。`False` を指定すると、この機能が無効化される（Python 3.12 で追加） |

`TemporaryDirectory` の属性:

| 属性 | 意味 |
|:---|:---|
| `name` | 一時ディレクトリのパスを表す文字列を格納する属性 |

`TemporaryDirectory` のメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:--:|
| `cleanup()` | 一時ディレクトリを中身のファイルごと削除する。処理中のエラーについては `ignore_cleanup_errors` 引数を参照 | `None` |
| `__enter__()` | `name` 属性を返す | `str` |
| `__exit__(exc, value, tb)` | `cleanup()` を呼び出す。Python 3.12 以降では、このオブジェクトが `delete` 引数に `False` を与えて生成されていた場合、このメソッドは何もしない | `None` |

In [ ]:
import tempfile
from pathlib import Path

path = None
with tempfile.TemporaryDirectory() as td:
    path = td
    print(f"{path=}")
    p = Path(path, "test.txt")
    p.touch()  # ファイルを作成
    print(f"{[x.name for x in Path(path).iterdir()]}")

assert not Path(path).exists()

path='/tmp/tmpydsbx3y0'
['test.txt']


### 低レベル関数 ###

`tempfile.TemporaryFile` などのクラスの内部で呼び出されている低レベル関数を直接使用することもできる。

``` python
tempfile.mkstemp(suffix=None, prefix=None, dir=None, text=False)
```

可能な限り最も安全な手段で一時ファイルを生成する。`tempfile.TemporaryFile()` と違って、`tempfile.mkstemp()` で作成された一時ファイルは、用済みになった時に手動で削除する必要がある（あるいは、システムによって一時ファイルが自動的に削除されることを期待することになる）。

引数の意味とデフォルトは、`tempfile.TemporaryFile()` のものと同じ。

``` python
tempfile.mkdtemp(suffix=None, prefix=None, dir=None)
```

可能な限り安全な方法で一時ディレクトリを作成する。`tempfile.TemporaryDirectory()` と違って、`tempfile.mkdtemp()` で作成された一時ディレクトリは、用済みになった時に手動で削除する必要がある（あるいは、システムによって一時ディレクトリが自動的に削除されることを期待することになる）。

引数の意味とデフォルトは、`tempfile.TemporaryDirectory()` のものと同じ。

``` python
tempfile.gettempdir()
```

一時ファイルに用いられるディレクトリの名前を返す。

この関数はモジュール内の全ての関数の `dir` 引数のデフォルト値を定義する。具体的には、以下の順番でディレクトリを検索する。

  1. 環境変数 `TMPDIR` で与えられているディレクトリ名。
  2. 環境変数 `TEMP` で与えられているディレクトリ名。
  3. 環境変数 `TMP` で与えられているディレクトリ名。
  4. プラットフォーム依存の場所。
  5. 現在の作業ディレクトリ。

この検索の結果はキャッシュされる。

In [1]:
import tempfile
tempfile.gettempdir()

'/tmp'

shutil
------

ファイルやディレクトリに対するコピー、移動、圧縮といった高水準の操作は、標準ライブラリの `shutil` モジュールを使う。

### ファイルのコピー ###

``` python
shutil.copyfileobj(fsrc, fdst[, length])
```

この関数は、ファイルオブジェクト `fsrc` の内容をファイルオブジェクト `fdst` にコピーする。戻り値は `None`。実装としては、`fsrc` の `read()` メソッドによる読み込みと `fdst` の `write()` メソッドによる書き込みの反復を、`length`（ `length` が与えられていない場合は `shutil.COPY_BUFSIZE`）サイズ単位で行っている。このため、ファイルのパーミッション（権限）とメタデータ（最終アクセス時間、最終変更時間など）はコピーしない。

`fsrc` と `fdst` には、`io.BytesIO` や `io.StringIO` も使える。

In [ ]:
import io
import shutil
src = io.StringIO("あいう")
dest_bin = io.BytesIO()
dest = io.TextIOWrapper(dest_bin, encoding="utf-8")
shutil.copyfileobj(src, dest)
dest.flush()
dest_bin.getvalue()

b'\xe3\x81\x82\xe3\x81\x84\xe3\x81\x86'

``` python
shutil.copyfile(src, dst, *, follow_symlinks=True)
```

この関数は、path-like オブジェクト `src` の内容を path-like オブジェクト `dst` にコピーし、`dst` を返す。実装としては、`src` を `open()` で開いて、返されたファイルオブジェクトを `copyfileobj()` に渡している。このため、ファイルのパーミッションとメタデータはコピーしない。

`dst` はファイルでなければならない。`dst` が既に存在するファイルを指定している場合、それは置き換えられる。`dst` が存在しない中間ディレクトリを含むとエラーが発生する。あらかじめ直上のディレクトリまで作成しておく必要がある。このような仕様は、実装が `open(src, 'rb')` と `open(dst, 'wb')` を呼び出していることによる。

`src` と `dst` が同じファイルだった場合、`shutil.SameFileError` 例外が発生する。

`src` がシンボリックリンクの場合、キーワード専用引数 `follow_symlinks` が偽なら `dst` はシンボリックリンクとして作成され、`follow_symlinks` が真なら `dst` には `src` のリンク先のファイルがコピーされる。

``` python
shutil.copymode(src, dst, *, follow_symlinks=True)
```

この関数は、パーミッションを `src` から `dst` にコピーする。最終アクセス時間、最終変更時間などのメタデータは一切コピーしない。戻り値は `None`。引数は、`shutil.copyfile()` と同じ。

``` python
shutil.copystat(src, dst, *, follow_symlinks=True)
```

この関数は、パーミッションを `src` から `dst` にコピーし、さらに最終アクセス時間、最終変更時間などの一部メタデータもコピーする。戻り値は `None`。引数は、`shutil.copyfile()` と同じ。

以下のメタデータはコピーされないことに注意する。

  * POSIX プラットフォーム: ACL、ファイルのオーナー、グループ
  * macOS: リソースフォーク（ファイルタイプ、クリエイター、アイコン、ウィンドウ位置、バージョン情報など）
  * Windows: 所有者、ACL、代替データストリーム（ADS）など

``` python
shutil.copy(src, dst, *, follow_symlinks=True)
```

この関数は、ファイルの内容とパーミッションを `src` から `dst` にコピーし、`dst` を返す。関数の内部では、 `shutil.copyfile()` と `shutil.copymode()` を呼び出している。ただし、`dst` 引数はそのままこれらの関数に渡されるのではなく、`dst` がディレクトリなら `dst` のパスと `src` のベース（パス要素の末尾）を連結した文字列に加工された上で渡される。したがって、**この関数は、ファイルをディレクトリにコピーすることができる**。

``` python
shutil.copy2(src, dst, *, follow_symlinks=True)
```

この関数は、ファイルのメタデータを保持しようとすることを除けば `copy()` と等価である。関数の内部では、`shutil.copyfile()` と `shutil.copystat()` を呼び出している。したがって、全てのメタデータがコピーされるわけではないことに注意する。

### ディレクトリのコピー ###

``` python
shutil.copytree(src, dst, symlinks=False, ignore=None, copy_function=copy2, ignore_dangling_symlinks=False, dirs_exist_ok=False)
```

この関数は、ディレクトリツリーを丸ごとコピーする。つまり、ディレクトリを配下のファイルやサブディレクトリも含めて再帰的にコピーする。戻り値は、`dst`。

| 引数 | 意味 |
|:---|:---|
| `src` | コピー対象のディレクトリを指定する。path-like オブジェクトが使える |
| `dst` | コピー後のディレクトリを指定する。path-like オブジェクトが使える |
| `symlinks` | `True` の場合、シンボリックリンクはコピー後もシンボリックリンクとなる。`False` の場合、リンク先のファイルそのものがコピーされる |
| `ignore` | 呼び出し可能オブジェクトを指定すると、そのオブジェクトが返す相対パス（カレントディレクトリを基準とする）の集合に含まれるディレクトリやファイルをコピー<br />しない。呼び出し可能オブジェクトは 2 つの引数を取るものでなければならず、第 1 引数にはディレクトリ、第 2 引数にはその中身のリストが渡される |
| `copy_function` | コピーに用いる関数を指定する。デフォルトでは `shutil.copy2()` が使用される |
| `ignore_dangling_symlinks` | `True` を指定すると、引数 `symlinks` が `False` のときにリンク先が存在しない場合でもエラーを出さないようになる |
| `dirs_exist_ok` | `False`（デフォルト）の場合、`dst` が既存のディレクトリならエラーが発生する。`True` を指定すると、`dst` が既存のディレクトリでもエラーにならない |

`ignore` 引数に渡す呼び出し可能オブジェクトは、次のユーティリティ関数で簡単に作成することができる。

``` python
shutil.ignore_patterns(*patterns)
```

このファクトリー関数は、 `shutil.copytree()` 関数の `ignore` 引数に渡すための呼び出し可能オブジェクトを作成する。`patterns` には glob パターンを複数指定でき、指定したパターンにマッチするファイル名およびディレクトリ名を含む相対パスの集合を返す呼び出し可能オブジェクトを作成できる。

以下のコードは、Colab が用意する sample_data ディレクトリを一時ディレクトリにコピーする。ただし、`shutil.ignore_patterns()` を使用して、拡張子 `.md` および `.json` のファイルをコピー対象から除外している。

In [ ]:
import tempfile
import shutil
from pathlib import Path

ignore = shutil.ignore_patterns("*.md", "*.json")
src = Path("./sample_data")
print(f"{ignore(src, [x.name for x in src.iterdir()])=}")

with tempfile.TemporaryDirectory() as td:
    dest = shutil.copytree(src, Path(td) / Path("sample_data"), ignore=ignore)
    print(f"{dest=}")
    print(f"{[x.name for x in Path(dest).iterdir()]}")

ignore(src, [x.name for x in src.iterdir()])={'README.md', 'anscombe.json'}
dest=PosixPath('/tmp/tmpp2k1s8co/sample_data')
['california_housing_train.csv', 'mnist_test.csv', 'mnist_train_small.csv', 'california_housing_test.csv']


### ファイル・ディレクトリの移動・削除 ###

``` python
shutil.move(src, dst, copy_function=copy2)
```

この関数は、ファイルまたはディレクトリ `src` を再帰的に別の場所 `dst` に移動して、`dst` を返す。`src` および `dst` は path-like オブジェクトが使える。`copy_function` にコピーに用いる関数を指定する。デフォルトでは shutil.copy2() が使用される。

`dst` が存在するディレクトリの場合、 `src` はそのディレクトリの中へ移動する。`dst` が存在していてそれがディレクトリでない場合、内部で `os.rename()` を使用するため、`os.rename()` と同じ結果になる。

``` python
shutil.rmtree(path, ignore_errors=False, onerror=None, *, onexc=None)
```

この関数は、ディレクトリを中身のファイルやサブディレクトリごと削除する。

| 引数 | 意味 |
|:---|:---|
| `path` | 削除するディレクトリを path-like オブジェクトで指定する |
| `ignore_errors` | `True` の場合、削除に失敗したことによるエラーは無視される。`False`（デフォルト）の場合、これらのエラーは `onerror` または `onexc` に指定した呼び出し可能オブジェクトで処<br />理され、両方が省略された場合は例外を送出する |
| `onerror` | `function`, `path`, `excinfo` の 3 つの引数をとる呼び出し可能オブジェクトを指定する。`function` には例外が発生した関数が、`path` には `function` に渡されたパスが、`excinfo` <br />には `sys.exc_info()` の戻り値が渡される。この引数は Python 3.12 で非推奨とされた |
| `onexc` | キーワード専用。Python 3.12 で追加。受け取る 3 番目の引数が発生した例外であることを除いて `onerror` と同様である |

### アーカイブ化操作 ###

``` python
shutil.make_archive(base_name, format[, root_dir[, base_dir[, verbose[, dry_run[, owner[, group[, logger]]]]]]])
```

この関数は、アーカイブファイルを作成してその名前を返す。主な引数は、次のとおり。

| 引数 | 意味 |
|:---|:---|
| `base_name` | 作成するアーカイブファイルの名前を指定する。拡張子は指定しないこと |
| `format` | アーカイブ方式を指定する。`'zip'`, `'tar'`, `'gztar'`, `'bztar'`, `'xztar'` を指定できる。`'zip'` の場合、GZIP 圧縮方式のデフォルトレベルが使用される |
| `root_dir` | アーカイブ対象となるディレクトリを指定する。デフォルトはカレントディレクトリ |
| `base_dir` | アーカイブを開始するディレクトリを、`root_dir` からの相対パスで指定する。デフォルトはカレントディレクトリ |

アーカイブに含まれる全てのパスは `root_dir` からの相対パスになる。

`root_dir` を指定して `base_dir` を指定しない場合、`root_dir` そのものではなくその中身が圧縮対象になる。たとえば、`root_dir=target_dir` とする場合、圧縮前と圧縮後のディレクトリ構成は次のように対応する。

``` text
（圧縮前）
current_dir
└target_dir --- root_dir
  ├─ sub_dir
  │   ├─ a.txt
  │   └─ b.txt
  ├─ c.txt
  └─ d.txt

（圧縮後）
sub_dir
├─ a.txt
└─ b.txt
c.txt
d.txt
```

`root_dir` と `base_dir` の両方を指定する場合、`base_dir` そのものとその中身が圧縮対象になる。たとえば、`root_dir='.', base_dir='target_dir'` とする場合、圧縮前と圧縮後のディレクトリ構成は次のように対応する。

``` text
（圧縮前）
current_dir --- root_dir
└target_dir --- base_dir
  ├─ sub_dir
  │   ├─ a.txt
  │   └─ b.txt
  ├─ c.txt
  └─ d.txt

（圧縮後）
target_dir
├─ sub_dir
│   ├─ a.txt
│   └─ b.txt
├─ c.txt
└─ d.txt
```

``` python
shutil.unpack_archive(filename[, extract_dir[, format[, filter]]])
```

この関数は、アーカイブをアンパックする。主な引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `filename` | アーカイブの絶対パスを path-like オブジェクトとして指定する |
| `extract_dir` | アーカイブをアンパックする先のディレクトリ名を path-like オブジェクトとして指定する。指定されなかった場合はカレントディレクトリが使われる |
| `format` | `filename` のアーカイブ形式を指定する。`zip`, `tar`, `gztar`, `bztar`, `xztar` を指定できる。指定を省略した場合、`filename` の拡張子から推測される |

zipfile
-------

`shutil.make_archive()` 関数と `shutil.unpack_archive()` 関数は、 ZIP 形式について標準ライブラリの `zipfile` パッケージを利用している。ZIP 形式について細やかな処理を行う場合は、`zipfile` パッケージを直接使用する必要がある。

`zipfile` パッケージが対応している圧縮方式は次のとおり。

| 圧縮方式 | 数値定数 | 使用するモジュール | 圧縮レベル |
|:---|:---|:---|:---|
| 非圧縮 | `zipfile.ZIP_STORED` | なし | なし |
| GZIP 圧縮方式 | `zipfile.ZIP_DEFLATED` | 標準ライブラリの `zlib` | 0 から 9 まで（`zlib` ではデフォルトのレベルが 6） |
| BZIP2 圧縮方式 | `zipfile.ZIP_BZIP2` | 標準ライブラリの `bz2` | 1 から 9 まで（`bz2` ではデフォルトのレベルが 9） |
| LZMA 圧縮方式 | `zipfile.ZIP_LZMA` | 標準ライブラリの `lzma` | なし |

この表では、圧縮方式は下に行くほど圧縮率が高く処理時間が長くなる。また、圧縮レベルは数字が大きいほど圧縮率が高く処理時間が長くなる。

`zipfile` パッケージが対応していない圧縮方式が使われている ZIP ファイルを読み取ろうとすると、`NotImplementedError` 例外が発生する。

``` python
zipfile.is_zipfile(filename)
```

この関数は、`filename` が ZIP ファイルの場合 `True` を返し、そうでない場合 `False` を返す。`filename` にはファイルのパス（文字列）かファイルオブジェクトを指定する。

In [ ]:
import zipfile
assert not zipfile.is_zipfile("./sample_data/README.md")

``` python
zipfile.ZipFile(file, mode='r', compression=ZIP_STORED, allowZip64=True, compresslevel=None, *, strict_timestamps=True, metadata_encoding=None)
```

`zipfile.ZipFile` は ZIP ファイルの読み書きのためのクラスである。コンストラクタの引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `file` | ファイルのパス（文字列）か、ファイルオブジェクト、 path-like オブジェクト |
| `mode` | `'r'`（ZIP ファイルの読み込み）, `'w'`（新規 ZIP ファイルの作成）, `'a'`（ZIP ファイルへの追加）, `'x'`（ファイルが存在しない場合にのみ ZIP ファイルを作成して書き込む）<br />を指定できる |
| `compression` | デフォルトで使用される圧縮方式を指定する。デフォルトでは非圧縮とされることに注意する |
| `allowZip64` | `True`（デフォルト）の場合、ZIP64 形式に対応する。ZIP64 形式は 4 GB を超えるファイルを扱える |
| `compresslevel` | デフォルトで使用される圧縮レベルを指定する。`None`（デフォルト）の場合、使用するモジュールのデフォルトのレベルとする |
| `strict_timestamps` | キーワード専用。`False` の場合、1980 年より古いタイムスタンプを 1980 年 1 月 1 日に設定し、2108 年以降のタイムスタンプを 2107 年 12 月 31 日に設定する |
| `metadata_encoding` | キーワード専用。Python 3.11 で追加。日本語のファイル名が文字化けするなら、Unix 系 OS では `'utf-8'` を指定し、Windows では `'cp932'` を指定すると解決する |

`zipfile.ZipFile` オブジェクトの主なメソッドは以下のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `namelist()` | ZIP ファイルに含まれる各ファイルの名前のリストを返す | `list` |
| `getinfo(name)` | ZIP ファイルに含まれる `name` という名前のファイルに関する情報を持つ `zipfile.ZipInfo` オブジェクトを返す。ZIP ファイルに含まれない<br />ファイル名に対して `getinfo()` を呼び出すと、`KeyError` が送出される | `ZipFile` |
| `infolist()` | ZIP ファイルに含まれる各ファイルに関する `zipfile.ZipInfo` オブジェクトからなるリストを返す | `list` |
| `read(name, pwd=None)` | ZIP ファイルに含まれるファイルの内容をバイト列で返す。`name` にはファイルの名前または `zipfile.ZipInfo` オブジェクトを指定する。ZIP<br /> ファイルが暗号化されている場合、`pwd` にパスワードを指定する必要がある | `bytes` |
| `extract(member, path=None, pwd=None)` | ZIP ファイルに含まれるファイルを指定して展開する。`member` にはファイルの名前または `zipfile.ZipInfo` オブジェクトを指定する。`path`<br /> には、展開先の別のディレクトリを指定する。`pwd` は暗号化された ZIP ファイルに使用されるパスワードを指定する | `None` |
| `extractall(path=None, members=None,`<br />` pwd=None)` | ZIP ファイルに含まれるファイルを全て展開する。`members` にファイルの名前のリストを指定した場合、`members` だけを展開する | `None` |
| `write(filename, arcname=None,`<br />` compress_type=None, compresslevel=None)` | 指定したファイルを ZIP ファイルにアーカイブする。`arcname` を指定するとその名前でアーカイブする。`compress_type` で圧縮方式を指定で<br />きる。`compresslevel` で圧縮レベルを指定できる | `None` |
| `writestr(zinfo_or_arcname, data,`<br />` compress_type=None, compresslevel=None)` | 文字列またはバイト列である `data` を `zinfo_or_arcname` で指定されるファイル名で ZIP ファイルにアーカイブする。`zinfo_or_arcname` に<br /> `zipfile.ZipInfo` オブジェクトを指定することができる。他のオプションは `write()` と同じ | `None` |
| `close()` | ZIP ファイルを閉じる | `None` |

`write()` メソッドや `writestr()` メソッドを使用する場合、アーカイブ中に使用するファイルの名前はアーカイブルートに対する相対パスでなければならない。言い換えると、その名前はパス区切り文字で始まってはいけない。

`zipfile.ZipFile` はコンテキストマネージャーにもなっているので、with 文をサポートする。次の例では、`myzip` は with 文のブロックが終了したときに、（たとえ例外が発生したとしても） `close()` メソッドが呼び出される:

``` python
with ZipFile('spam.zip', 'w') as myzip:
    myzip.write('eggs.txt')
```

`write()` メソッドは複数のファイルやディレクトリを一度に圧縮することはでできないので、for 文での繰り返しにより 1 つずつファイルを追記モード（`mode='a'`）でアーカイブ化する必要がある。このときにディレクトリ構成を維持したいなら、`arcname` を相対パスで指定する必要がある。

`zipfile.ZipInfo` オブジェクトの主な属性は次のとおり。

| 属性 | 意味 |
|:---|:---|
| `filename` | アーカイブ中のファイルの名前 |
| `date_time` | アーカイブ中のファイルの最終更新日時。西暦年（>= 1980）、月、日、時、分、秒を示す 6 要素タプル |
| `compress_size` | 圧縮後のデータのサイズ |
| `file_size` | 圧縮前のファイルのサイズ |

`zipfile` パッケージはコマンドラインインターフェースを提供している。

ZIP アーカイブを新規に作成したい場合、`-c` オプションの後にまとめたいファイルを列挙する:

``` shell
python -m zipfile -c monty.zip spam.txt eggs.txt
```

ディレクトリを渡すこともできる:

``` shell
python -m zipfile -c monty.zip mywork/
```

ZIP アーカイブを特定のディレクトリに展開したい場合、`-e` オプションを使用する:

``` shell
python -m zipfile -e monty.zip target-dir/
```

ZIP アーカイブ内のファイル一覧を表示するには `-l` オプションを使用する:

``` shell
python -m zipfile -l monty.zip
```

tarfile
-------

`shutil.make_archive()` 関数と `shutil.unpack_archive()` 関数は、  TAR 形式について標準ライブラリの `tarfile` モジュールを利用している。TAR 形式について細やかな処理を行う場合は、`tarfile` モジュールを直接使用する必要がある。

``` python
tarfile.is_tarfile(name)
```

この関数は、`name` が TAR 形式の場合 True を返し、そうでない場合 False を返す。`name` にはファイルのパス（文字列）かファイルオブジェクトを指定する。

In [ ]:
import tarfile
assert not tarfile.is_tarfile("./sample_data/README.md")

``` python
tarfile.open(name=None, mode='r', fileobj=None, bufsize=10240, **kwargs)
```

この関数は、 `tarfile.TarFile` オブジェクトを返す。主な引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `name` | パス名。path-like object を使用できる |
| `mode` | `'filemode:compression'` の形式をとる文字列を指定する。`:compression` は省略可能。`mode` の指定を省略した場合は非圧縮で読み込み用に開く<br /><br />・filemode:  `'r'`（読み込み）, `'w'`（新規作成）, `'a'`（追加）, `'x'`（既存でなければ新規作成）を指定できる<br /><br />・compression: `'gz'`（GZIP 圧縮）、`'bz2'`（BZIP2 圧縮）、`'xz'`（LZMA 圧縮）を指定できる。compression を省略した場合は非圧縮となる |

たとえば、`mode='w:gz'` の場合、GZIP 圧縮で書き込み用に開く。

`tarfile.TarFile` オブジェクトの主なメソッドは次のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `getnames()` | TAR ファイル内にアーカイブされているファイルの名前のリストを返す | `list` |
| `getmember(name)` | TAR ファイル内にアーカイブされている `name` という名前のファイルに関する情報を持つ `tarfile.TarInfo` オブジェクトを返す | `TarInfo` |
| `getmembers()` | TAR ファイル内にアーカイブされている各ファイルに関する `tarfile.TarInfo` オブジェクトのリストを返す | `list` |
| `extractall(path='.', members=None,`<br />` *, numeric_owner=False, filter=None)` | アーカイブをカレントディレクトリに展開する。`path` に展開先のディレクトリを指定できる。`members` に `tarfile.TarInfo` オブ<br />ジェクトのリストを指定した場合、`members` だけを展開する | `None` |
| `extractfile(member)` | アーカイブから `member` をファイルオブジェクトとして抽出する。`member` はファイル名か `tarfile.TarInfo` オブジェクト | ファイルオブジェクト |
| `add(name, arcname=None, recursive=True,`<br />` *, filter=None)` | ファイル `name` を TAR ファイルに追加してアーカイブする。`arcname` を指定するとその名前でアーカイブする。`name` がディレク<br />トリの場合、`recursive` が `True`（デフォルト）なら再帰的に追加される | `None` |
| `addfile(tarinfo, fileobj=None)` | `TarInfo` オブジェクト `tarinfo` をアーカイブに追加する | `None` |
| `close()` | TAR ファイルを閉じる | `None` |

`tarfile.TarFile` はコンテキストマネージャーにもなっているので、with 文をサポートする。次の例では、`mytar` は with 文のブロックが終了したときに、（たとえ例外が発生したとしても） `close()` メソッドが呼び出される:

``` python
with tarfile.open("archived.tar.gz", mode="w:gz") as mytar:
    for name in ["foo.txt", "bar.txt", "quux.txt"]:
        mytar.add(name)
```

`tarfile.TarInfo` オブジェクトの主な属性は次のとおり。

| 属性 | 意味 |
|:---|:---|
| `name` | アーカイブ中のファイルの名前 |
| `mtime` | アーカイブ中のファイルの最終更新日時となるエポック秒 |
| `size` | ファイルサイズ |

`tarfile ` モジュールはコマンドラインインターフェースを提供している。

TAR アーカイブを新規に作成したい場合、`-c` オプションの後にまとめたいファイルを列挙する:

``` shell
python -m tarfile -c monty.tar spam.txt eggs.txt
```

ディレクトリを渡すこともできる:

``` shell
python -m tarfile -c monty.tar mywork/
```

TAR アーカイブをカレントディレクトリに展開したい場合、`-e` オプションを使用する:

``` shell
python -m tarfile -e monty.tar
```

ディレクトリ名を渡すことで TAR アーカイブを別のディレクトリに取り出すこともできる:

``` python
python -m tarfile -e monty.tar other-dir/
```

TAR アーカイブ内のファイル一覧を表示するには `-l` オプションを使用する:

``` shell
python -m tarfile -l monty.tar
```